In [3]:
import os
os.sys.path.append('F:\Installed Softwares\Anaconda\envs\opencv-env\Lib\site-packages')
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input, decode_predictions

F:\Installed Softwares\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\roval\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\roval\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\roval\AppData\Roami

In [2]:
import os
os.sys.path.append('F:\Installed Softwares\Anaconda\envs\opencv-env\Lib\site-packages')
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input, decode_predictions

WIDTH=299
HEIGHT=299
BATCH_SIZE=64
test_dir = '../Dataset/test/'
train_dir = '../Dataset/train/'
val_dir = '../Dataset/val/'

print("\nTraining Data Set")
train_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
train_flow = train_generator.flow_from_directory(train_dir, target_size=(HEIGHT, WIDTH),  batch_size = BATCH_SIZE)

print("\nValidation Data Set")
val_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
val_flow = val_generator.flow_from_directory(val_dir,target_size=(HEIGHT, WIDTH),batch_size = BATCH_SIZE)

print("\nTest Data Set")
test_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
test_flow = test_generator.flow_from_directory( test_dir,target_size=(HEIGHT, WIDTH),batch_size = BATCH_SIZE)


Training Data Set
Found 1458 images belonging to 4 classes.

Validation Data Set
Found 207 images belonging to 4 classes.

Test Data Set
Found 421 images belonging to 4 classes.


In [3]:
from keras.models import Sequential, Model, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger
from keras import optimizers, models
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras import applications
from keras import backend as K
import tensorflow as tf
import os

NUM_PARALLEL_EXEC_UNITS = 8

config = tf.ConfigProto( intra_op_parallelism_threads=NUM_PARALLEL_EXEC_UNITS,inter_op_parallelism_threads=1)

session = tf.Session(config=config)
K.set_session(session)

os.environ["OMP_NUM_THREADS"] = str(NUM_PARALLEL_EXEC_UNITS)
os.environ["KMP_BLOCKTIME"] = "1"
os.environ["KMP_SETTINGS"] = "1"
os.environ["KMP_AFFINITY"]= "granularity=fine,verbose,compact,1,0"

In [4]:
base_model = applications.InceptionV3(weights='imagenet', include_top=False,input_shape=(WIDTH, HEIGHT,3))

x = base_model.output

x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_flow.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=optimizers.Adam(lr=0.001), metrics=['accuracy', 'top_k_categorical_accuracy'], loss='categorical_crossentropy')
model.summary()



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________

In [6]:
import math
top_layers_file_path="top_layers.iv3.hdf5"

checkpoint = ModelCheckpoint(top_layers_file_path, monitor='loss', verbose=1, save_best_only=True, mode='min')
tb = TensorBoard(log_dir='./logs', batch_size=val_flow.batch_size, write_graph=True, update_freq='batch')
early = EarlyStopping(monitor="loss", mode="min", patience=5)
csv_logger = CSVLogger('./logs/iv3-log.csv', append=True)

history = model.fit_generator(train_flow, epochs=1, verbose=1,validation_data=val_flow,validation_steps=math.ceil(val_flow.samples/val_flow.batch_size),steps_per_epoch=math.ceil(train_flow.samples/train_flow.batch_size),callbacks=[checkpoint, early, tb, csv_logger])

Epoch 1/1
23/23 [==============================] - 397s 17s/step - loss: 0.7893 - accuracy: 0.6639 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.8173 - val_accuracy: 0.5266 - val_top_k_categorical_accuracy: 1.0000

Epoch 00001: loss improved from inf to 0.79071, saving model to top_layers.iv3.hdf5


In [7]:
model.load_weights(top_layers_file_path)
loss, acc, top_5 = model.evaluate_generator(test_flow,verbose = True,steps=math.ceil(test_flow.samples/test_flow.batch_size))
print("Loss: ", loss)
print("Acc: ", acc)
print("Top 5: ", top_5)

7/7 [==============================] - 109s 16s/step
Loss:  1.0595402717590332
Acc:  0.489311158657074
Top 5:  1.0


In [ ]:
label = [k for k,v in train_flow.class_indices.items()]
with open('iv3-labels.txt', 'w+') as file:file.write("\n".join(label))

In [ ]:
from keras.preprocessing import image
import numpy as np
import glob
import random

file_list = glob.glob("../Dataset/test/*/*")
img_path = random.choice(file_list)
img_cat = os.path.split(os.path.dirname(img_path))[1]
print("Image Category: ", img_cat)
img = image.load_img(img_path, target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
print("Raw Predictions: ", preds)

top_x = 3
top_args = preds[0].argsort()[-top_x:][::-1]
preds_label = [label[p] for p in top_args]
print("\nTop " + str(top_x) + " confidence: " + " ".join(map(str, sorted(preds[0])[-top_x:][::-1])))
print("Top " + str(top_x) + " labels: " + " ".join(map(str, preds_label)))

In [ ]:
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io

input_model_path = top_layers_file_path
output_model_name = "top_layers.iv3.pb"
output_model_dir = "tf_model"

K.set_learning_phase(0)
sess = K.get_session()

test_model = models.load_model(input_model_path)
orig_output_node_names = [node.op.name for node in test_model.outputs]

constant_graph = graph_util.convert_variables_to_constants(sess,sess.graph.as_graph_def(),orig_output_node_names)
graph_io.write_graph(constant_graph,output_model_dir,output_model_name,as_text=False)